# FOOD FOR ALL - Analytics Application

**STEP 1:- Scan news collection from Watson Discovery service using Discovery API**. 

Watson discovery is an enterprise search engine that can comb through an unstructured "document collection" and retieve the relevant information based on a search query criteria. The sample query string for this notebook used is "migrant labourers india". The results of the query are in the JSON format with relevant snapshot passages and the link to the news portal along with date. The following piece of code calls the API on a discovery service that we have instantiated hosted on IBM Cloud The code  parses the JSON data to list out the data in a tabular form.

In [1]:
import json
import subprocess
import sys
subprocess.check_call([sys.executable, "-m", "pip", "install", "ibm_watson"])
from ibm_watson import DiscoveryV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator('ZpcfFyTpX2_TLS667RZ5UJD-wWaJhtgCEc1EADliWLfi')
discovery = DiscoveryV1(
    version='2020-05-31',
    authenticator=authenticator
)
#Set the URL to our discovery instance
discovery.set_service_url('https://api.eu-gb.discovery.watson.cloud.ibm.com/instances/22aecf78-2273-4e59-acec-38635eee713e')

#Query the Discovery service to fetch the data related to migrants
out_str = discovery.query('system', 'news-en', query='enriched_text.concepts.text:migrant india').get_result()


#print(out_str['matching_results'])

for i in out_str['results']:
    #print(json.dumps(i, indent=2) )
    print("Date of incident: " + i['crawl_date'])
    print("Source of news: " + i['url'])
    print("Actual content: " + i['text'])
    print("=============================================")

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200606103029-0000
KERNEL_ID = d61cf881-8372-4029-9903-7eabd81c2979
Date of incident: 2020-05-05T06:57:26Z
Source of news: https://savedelete.com/photos/bihar-migrant-workers-return-to-patna-from-ernakulam-via-a-special-shramik-train-during-the-extended-nationwide-lockdown/311003/
Actual content: Kolkata: Bihar migrant workers return to Patna from Ernakulam via a Special Shramik train during the extended nationwide lockdown, on May 4, 2020. (Photo: IANS) A 1000-bed facility being contructed at the exhibition ground in Mumbai’s Bandra Kurla Complex to provide quarantine and isolation facilities for non-critical COVID-19 patients during
Date of incident: 2020-05-28T13:09:43Z
Source of news: http://www.indiaenvironmentportal.org.in/content/467701/workers-in-the-time-of-covid-19-evidence-from-a-rapid-assessment-in-bihar/
Actual content: India.The study and this report seeks to rapidly discover and brief

**STEP2:- Get latitude and longitude coordinates of location that needs help.**  

For purpose of demo, we have manually extracted the named entities of interest to the use case. In the long term, we plan to use NLP and Text Analytics to do extraction automatically. So for the location "HSR Layout Sector 4" from Step1 above, we use GeoPositionalAPI to get the exact coordinates. The API takes a text string location and returns the latitude and longitude.

In [14]:
import pycurl
from io import BytesIO
buffer = BytesIO()
c = pycurl.Curl()
c.setopt(c.URL, 'http://api.positionstack.com/v1/forward?access_key=3e8d5380120b0106dcf33945dfda5866&query=Hsr+laylout+Sector+4')
c.setopt(c.WRITEDATA, buffer)
c.perform()
c.close()
body = buffer.getvalue()
# Body is a byte string.
# We have to know the encoding in order to print it to a text file
# such as standard output.
print(body.decode('iso-8859-1'))

{"data":[{"latitude":12.914749,"longitude":77.638341,"type":"address","name":"No.3 2nd Floor, Archanaa, Opp To Bdm Complex, 4Th Sector HSR Layout, Sector 4, HSR Layout","number":"No.3 2nd Floor, Archanaa, Opp To Bdm Complex, 4Th Sector","postal_code":"560102","street":"HSR Layout, Sector 4, HSR Layout","confidence":1,"region":"Karnataka","region_code":"KA","county":"Bangalore","locality":"Bangalore","administrative_area":null,"neighbourhood":null,"country":"India","country_code":"IND","continent":"Asia","label":"No.3 2nd Floor, Archanaa, Opp To Bdm Complex, 4Th Sector HSR Layout, Sector 4, HSR Layout, Bangalore, India"},{"latitude":12.907437,"longitude":77.647266,"type":"venue","name":"HSR 2nd Sector Park","number":null,"postal_code":null,"street":null,"confidence":1,"region":"Karnataka","region_code":"KA","county":"Bangalore","locality":"Bangalore","administrative_area":null,"neighbourhood":null,"country":"India","country_code":"IND","continent":"Asia","label":"HSR 2nd Sector Park, Ba

**STEP3:- Read the NGO/Foundation data from Cloud Object Storage into a DataFrame.**

We have used COS as a data storage for the purpose of demo. A CSV file containing the names; lat/long info of NGOs is read into DataFrame (ngo_df)- so that it can used further for geo spatial analytics.

In [3]:
from pyst import STContext
# Register STContext, which is the main entry point
stc = STContext(spark.sparkContext._gateway)

In [4]:
# Read ngo csv file
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_f27f0e76464c443c9c6ab99d6ad324d8 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='XmpNPoFEGKr9FRHUajO-8BnyMRUKFOU36FXnEgOaPmYA',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_f27f0e76464c443c9c6ab99d6ad324d8.get_object(Bucket='foodforall-donotdelete-pr-rnf4ycyspogcjz',Key='NGO.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

ngo_df = pd.read_csv(body)
ngo_df

,name,city,latitude,longitude
0,Akshaya Patra Foundation,Bengaluru,13.017672,77.549820
1,the Attria Foundation,Bengaluru,13.032518,77.592124
2,Humanity First,Hyderabad,17.385044,78.486671
3,Habitat for Humanity India,Bengaluru,13.026731,77.631395
4,Oxfam India,Bengaluru,13.016299,77.636412
5,SEEDS,New Delhi,28.588800,77.084866
6,Humane Universal Good Deeds Network,Bengaluru,12.912118,77.644555
7,Rise Against Hunger (RAHI),Bengaluru,13.018392,77.653632


**STEP4:- Read the location data from Cloud Object Storage into a DataFrame.**

We have used COS as a data storage for the purpose of demo. A CSV file containing the names; lat/long info of locations (that are seeking aid) is read into DataFrame (migrant_df) - so that it can used further for geo spatial analytics.

In [5]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_3d68c5c911bc498ba98b556479cc036a = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='pVIo94tFckAgwVsMBQXRQgJMhwX2yuLegRTUi8U5m6Kj',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_3d68c5c911bc498ba98b556479cc036a.get_object(Bucket='projectsurbhi-donotdelete-pr-iywkvi1mn5xqjc',Key='migrant.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

migrant_df = pd.read_csv(body)
migrant_df.head()

,city,Location,latitude,longitude,supplies
0,Bengaluru,HSR Layout Sector 7,12.916340,77.649467,105
1,Bengaluru,HSR Layout Sector 1,12.916140,77.623459,119
2,Bengaluru,Shanti Nagar,12.989158,77.606089,50
3,Bengaluru,Yeshwantpur,13.023830,77.552921,550
4,Bengaluru,Mahalaxmi Layout,13.012419,77.543993,490


In [6]:
# Convert pandas dataframe to geo dataframe for NGO
import csv, json
from geojson import Feature, FeatureCollection, Point
features = []
for index, row in ngo_df.iterrows():
    latitude, longitude = map(float, (row['latitude'], row['longitude']))
    features.append(
        Feature(
            geometry = Point((longitude, latitude)),
            properties = {
                'name': row['name'],
                 'city': row['city'],
                'latitude': row['latitude'],
                 'longitude': row['longitude']
            }
        )
    )
ngo_json = FeatureCollection(features)
ngo_df = stc.geojson_reader().read(ngo_json)
ngo_df

,city,latitude,longitude,name,geometry
0,Bengaluru,13.017672,77.549820,Akshaya Patra Foundation,"Point(13.017672, 77.54982)"
1,Bengaluru,13.032518,77.592124,the Attria Foundation,"Point(13.032518, 77.592124)"
2,Hyderabad,17.385044,78.486671,Humanity First,"Point(17.385044, 78.486671)"
3,Bengaluru,13.026731,77.631395,Habitat for Humanity India,"Point(13.026731, 77.631395)"
4,Bengaluru,13.016299,77.636412,Oxfam India,"Point(13.016299, 77.63641199999999)"
5,New Delhi,28.588800,77.084866,SEEDS,"Point(28.5888, 77.084866)"
6,Bengaluru,12.912118,77.644555,Humane Universal Good Deeds Network,"Point(12.912118, 77.64455500000001)"
7,Bengaluru,13.018392,77.653632,Rise Against Hunger (RAHI),"Point(13.018392, 77.653632)"


In [7]:
# Convert pandas dataframe to geo dataframe for NGO
features = []
for index, row in migrant_df.iterrows():
    latitude, longitude = map(float, (row['latitude'], row['longitude']))
    features.append(
        Feature(
            geometry = Point((longitude, latitude)),
            properties = {
                'Location': row['Location'],
                 'city': row['city'],
                'latitude': row['latitude'],
                 'longitude': row['longitude']
            }
        )
    )
migrant_json = FeatureCollection(features)
migrant_df = stc.geojson_reader().read(migrant_json)
migrant_df

,Location,city,latitude,longitude,geometry
0,HSR Layout Sector 7,Bengaluru,12.916340,77.649467,"Point(12.91634, 77.649467)"
1,HSR Layout Sector 1,Bengaluru,12.916140,77.623459,"Point(12.916139999999999, 77.623459)"
2,Shanti Nagar,Bengaluru,12.989158,77.606089,"Point(12.98915835, 77.60608918)"
3,Yeshwantpur,Bengaluru,13.023830,77.552921,"Point(13.02383, 77.5529215)"
4,Mahalaxmi Layout,Bengaluru,13.012419,77.543993,"Point(13.01241878, 77.543993)"
5,Yelahanka,Bengaluru,13.078474,77.606894,"Point(13.0784743, 77.6068938)"
6,Mubarak Nagar,Delhi,28.575500,77.227400,"Point(28.5755, 77.2274)"
7,Vasant Vihar,Delhi,28.560802,77.161174,"Point(28.5608021, 77.1611738)"
8,Chandani Cahwk,Delhi,28.656000,77.231000,"Point(28.656, 77.23100000000001)"


**STEP5:- Find nearest NGOs for the location that needs help.**

This is the most crucial step of the analytics piece. We want food to be delivered fresh and as quickly as possible to the location. For this to work, we need to notify to NGOs that is geographically near to the location. We use spatio temporal functions to find the nearest neighbors. For purpose of demo we find the NGOs within 7KMs radius of the location. And the result shows there 6 NGOs nearest to "Shanti Nagar" location in Bangalore. 

In [8]:
tile_size = 100000 #in meters
si = stc.tessellation_index(tile_size=tile_size) # we leave bbox as None to use full earth as boundingbox
si.from_df(ngo_df, 'name', 'geometry', verbosity='error') #Populate the spatial index

nearest_ngo_name=[]
for row in migrant_df.itertuples():
    loc=row.geometry

    #Print NGO's with 5km radius of given point
    nearest_ngo = si.within_distance_with_info(loc, 7000)
    nearest_points=[]
    for i in nearest_ngo: 
        nearest_points.append(i[0])
    nearest_ngo_name.append(nearest_points)
# Add new column with names of NGO within the range
migrant_df['NGO within range'] = nearest_ngo_name
migrant_df

8 entries processed, 8 entries successfully added


,Location,city,latitude,longitude,geometry,NGO within range
0,HSR Layout Sector 7,Bengaluru,12.916340,77.649467,"Point(12.91634, 77.649467)",[Humane Universal Good Deeds Network]
1,HSR Layout Sector 1,Bengaluru,12.916140,77.623459,"Point(12.916139999999999, 77.623459)",[Humane Universal Good Deeds Network]
2,Shanti Nagar,Bengaluru,12.989158,77.606089,"Point(12.98915835, 77.60608918)","[Habitat for Humanity India, Akshaya Patra Fou..."
3,Yeshwantpur,Bengaluru,13.023830,77.552921,"Point(13.02383, 77.5529215)","[Akshaya Patra Foundation, the Attria Foundation]"
4,Mahalaxmi Layout,Bengaluru,13.012419,77.543993,"Point(13.01241878, 77.543993)","[Akshaya Patra Foundation, the Attria Foundation]"
5,Yelahanka,Bengaluru,13.078474,77.606894,"Point(13.0784743, 77.6068938)","[Habitat for Humanity India, the Attria Founda..."
6,Mubarak Nagar,Delhi,28.575500,77.227400,"Point(28.5755, 77.2274)",[]
7,Vasant Vihar,Delhi,28.560802,77.161174,"Point(28.5608021, 77.1611738)",[]
8,Chandani Cahwk,Delhi,28.656000,77.231000,"Point(28.656, 77.23100000000001)",[]


**STEP5:- Visualization - Part1.**

A picture can speak a thousand words. A visualization chart can show what the code wants to say in a snap. The following lines of code plots the locations on the map to show the total number of locations needing help and NGOs that offer help. This helps to make out clusters that are sparsely spread and ones that need more attention. The NGO locations are marked in RED and migrant locations are marked in BLUE

In [9]:
import folium
m1 = folium.Map([13.01767,77.54990])

locs_ngo = zip(ngo_df.latitude, ngo_df.longitude)
locs_migrants = zip(migrant_df.latitude, migrant_df.longitude)
for location in locs_ngo:
    folium.CircleMarker(location=location, color="red",   radius=4).add_to(m1)
for location in locs_migrants:
    folium.CircleMarker(location=location, color="blue",   radius=4).add_to(m1)

m1

**STEP6:- Visualization-Part2** 

The following lines of code plots the connections between NGOs and locations. This way we know on the India map, the nearest NGOs for a location again visually representing the crux of analytics piece. This data is then fed to a database. The FoodForAll system can then pick the data from the database to display on the web portal.

In [10]:
from geojson import MultiLineString,LineString

nearest_points=[]
for row in migrant_df.itertuples():
    loc=row.geometry
    nearest_ngo = si.within_distance_with_info(loc, 7000)
    for i in nearest_ngo: 
        point1=[]
        temp=[]
        point2=[]
        point1.append(loc.get_longitude())
        point1.append(loc.get_latitude())
        point2.append(i[1].get_longitude())
        point2.append(i[1].get_latitude())
        temp.append(point1)
        temp.append(point2)
        nearest_points.append(temp)
migrant_df['ngo_geo'] = migrant_df.apply(lambda row: MultiLineString(nearest_points), axis=1)
migrant_df



,Location,city,latitude,longitude,geometry,NGO within range,ngo_geo
0,HSR Layout Sector 7,Bengaluru,12.916340,77.649467,"Point(12.91634, 77.649467)",[Humane Universal Good Deeds Network],"{'type': 'MultiLineString', 'coordinates': [[[..."
1,HSR Layout Sector 1,Bengaluru,12.916140,77.623459,"Point(12.916139999999999, 77.623459)",[Humane Universal Good Deeds Network],"{'type': 'MultiLineString', 'coordinates': [[[..."
2,Shanti Nagar,Bengaluru,12.989158,77.606089,"Point(12.98915835, 77.60608918)","[Habitat for Humanity India, Akshaya Patra Fou...","{'type': 'MultiLineString', 'coordinates': [[[..."
3,Yeshwantpur,Bengaluru,13.023830,77.552921,"Point(13.02383, 77.5529215)","[Akshaya Patra Foundation, the Attria Foundation]","{'type': 'MultiLineString', 'coordinates': [[[..."
4,Mahalaxmi Layout,Bengaluru,13.012419,77.543993,"Point(13.01241878, 77.543993)","[Akshaya Patra Foundation, the Attria Foundation]","{'type': 'MultiLineString', 'coordinates': [[[..."
5,Yelahanka,Bengaluru,13.078474,77.606894,"Point(13.0784743, 77.6068938)","[Habitat for Humanity India, the Attria Founda...","{'type': 'MultiLineString', 'coordinates': [[[..."
6,Mubarak Nagar,Delhi,28.575500,77.227400,"Point(28.5755, 77.2274)",[],"{'type': 'MultiLineString', 'coordinates': [[[..."
7,Vasant Vihar,Delhi,28.560802,77.161174,"Point(28.5608021, 77.1611738)",[],"{'type': 'MultiLineString', 'coordinates': [[[..."
8,Chandani Cahwk,Delhi,28.656000,77.231000,"Point(28.656, 77.23100000000001)",[],"{'type': 'MultiLineString', 'coordinates': [[[..."


In [11]:
!pip install geopandas
import geopandas as gpd
line_gdf=gpd.GeoDataFrame(migrant_df, geometry=migrant_df.ngo_geo, crs={"init":"EPSG:4326"})
line_gdf)
folium.GeoJson(line_gdf).add_to(m1)
m1

    100% |################################| 931kB 5.6MB/s eta 0:00:01
    100% |################################| 10.9MB 1.5MB/s eta 0:00:01
    100% |################################| 14.7MB 1.3MB/s eta 0:00:01
    100% |################################| 10.1MB 1.3MB/s eta 0:00:01
    100% |################################| 1.8MB 4.9MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/ee/ff/48bde5c0f013094d729fe4b0316ba2a24774b3ff1c52d924a8a4cb04078a/six-1.15.0-py2.py3-none-any.whl
    100% |################################| 92kB 6.6MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl
    100% |################################| 20.1MB 926kB/s eta 0:00:01                | 266kB 55.3MB/s eta 0:00:01
    100% |################################| 512kB 5.9MB/s eta 0:00:01
tensorflow 1.13.1 requires tensorboard<1.14.0,>=1.13.0, which is not insta

/home/spark/shared/user-libs/python3.6/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
